In [1]:
import pandas as pd

In [20]:
import credentials as cred
key = cred.mapquest_api_key

In [3]:
import pandas as pd

url = "https://public.opendatasoft.com/explore/dataset/us-county-boundaries/download/?format=csv&refine.statefp=06&location=2,40.61349,40.02538&timezone=America/Los_Angeles&lang=en&use_labels_for_header=true&csv_separator=%3B"

df = pd.read_csv(url, sep=";")
new_df = df[["NAME", "Geo Shape"]].copy()

new_df.head()

,NAME,Geo Shape
0,Kern,"{""coordinates"": [[[-119.913659, 35.439262], [-..."
1,Kings,"{""coordinates"": [[[-119.958925, 36.255468], [-..."
2,San Benito,"{""coordinates"": [[[-121.483002, 36.76505], [-1..."
3,Fresno,"{""coordinates"": [[[-120.422187, 36.840139], [-..."
4,Plumas,"{""coordinates"": [[[-121.367008, 40.077673], [-..."


In [4]:
len(new_df)

58

In [5]:
state_bounds = pd.read_csv("https://gist.githubusercontent.com/a8dx/2340f9527af64f8ef8439366de981168/raw/81d876daea10eab5c2675811c39bcd18a79a9212/US_State_Bounding_Boxes.csv")
state_bounds = state_bounds[['NAME', 'STUSPS', 'xmin', 'ymin', 'xmax', 'ymax']]
state_bounds = state_bounds.rename(columns={"xmin": "min_lng", "xmax": "max_lng", "ymin": "min_lat", "ymax": "max_lat"})

In [6]:
import ast
import pandas as pd

# Function to extract latitude and longitude from coordinates
def extract_coordinates(geo_shape):
    geo_dict = ast.literal_eval(geo_shape)
    coordinates = []

    if geo_dict['type'] == 'Polygon':
        coordinates = geo_dict['coordinates'][0]
    elif geo_dict['type'] == 'MultiPolygon':
        for polygon in geo_dict['coordinates']:
            coordinates.extend(polygon[0])
    elif geo_dict['type'] == 'Point':
        coordinates = [geo_dict['coordinates']]

    latitudes = [coord[1] for coord in coordinates]
    longitudes = [coord[0] for coord in coordinates]
    return latitudes, longitudes

# Create a list to store county bounds dataframes
county_bounds_list = []

# Iterate over each county
for _, row in new_df.iterrows():
    county_name = row['NAME']
    latitudes, longitudes = extract_coordinates(row['Geo Shape'])

    # Calculate the minimum and maximum latitude and longitude for the county
    min_latitude = min(latitudes)
    max_latitude = max(latitudes)
    min_longitude = min(longitudes)
    max_longitude = max(longitudes)

    # Create a dataframe for the county bounds
    county_bounds_df = pd.DataFrame({'County': county_name,
                                     'Min_Latitude': min_latitude,
                                     'Max_Latitude': max_latitude,
                                     'Min_Longitude': min_longitude,
                                     'Max_Longitude': max_longitude}, index=[0])

    # Append the county bounds dataframe to the list
    county_bounds_list.append(county_bounds_df)

# Concatenate all county bounds dataframes into a single dataframe
county_bounds = pd.concat(county_bounds_list, ignore_index=True)

# Sort the DataFrame by county name
county_bounds = county_bounds.sort_values('County')

# Format the DataFrame for better display
county_bounds_styled = county_bounds.style.hide_index().set_caption('County Bounds').format({
    'Min Latitude': '{:.6f}',
    'Max Latitude': '{:.6f}',
    'Min Longitude': '{:.6f}',
    'Max Longitude': '{:.6f}'
})

# Display the formatted DataFrame
# display(county_bounds_styled)



C:\Users\gansi\AppData\Local\Temp\ipykernel_5156\3874296136.py:52: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  county_bounds_styled = county_bounds.style.hide_index().set_caption('County Bounds').format({


In [7]:
county_bounds.head(5)

,County,Min_Latitude,Max_Latitude,Min_Longitude,Max_Longitude
22,Alameda,37.454186,37.905824,-122.373782,-121.469214
34,Alpine,38.326880,38.933324,-120.073331,-119.542332
14,Amador,38.217951,38.709029,-121.027710,-120.072333
57,Butte,39.295621,40.152024,-122.069431,-121.076695
12,Calaveras,37.831858,38.509869,-120.995500,-120.019951


In [8]:
import sqlite3

In [9]:
conn = sqlite3.connect('test.db')

In [10]:
county_bounds.to_sql("counties", conn, if_exists="replace", index=False)

58

In [12]:
county_bounds[county_bounds['County'] == 'Los Angeles']['Min_Latitude']

39    32.75004
Name: Min_Latitude, dtype: float64

In [13]:
import traffic_data as td

def get_county_incidents(conn, county):
    cmd =\
    f"""
    SELECT Min_Latitude, Max_Latitude, Min_Longitude, Max_Longitude FROM counties
    WHERE County LIKE '{county.upper()}'
    """
    # print(cmd)
    cursor = conn.cursor()
    cursor.execute(cmd)
    # min_lat, max_lat, min_lng, max_lng = cursor.fetchone()
    bbox = cursor.fetchone()

    return td.get_incidents_in_area(conn, bbox)

def update_county_incidents(conn, key, county):
    cmd =\
    f"""
    SELECT Min_Latitude, Max_Latitude, Min_Longitude, Max_Longitude FROM counties
    WHERE County LIKE '{county.upper()}'
    """
    # print(cmd)
    cursor = conn.cursor()
    cursor.execute(cmd)
    # min_lat, max_lat, min_lng, max_lng = cursor.fetchone()
    bbox = cursor.fetchone()
    td.store_traffic_data(conn, key, bbox)

def get_counties(conn):
    cmd =\
    """
    SELECT county FROM counties
    """
    cursor = conn.cursor()
    cursor.execute(cmd)
    return [i[0] for i in cursor.fetchall()]


In [14]:
get_counties(conn)

['Alameda',
 'Alpine',
 'Amador',
 'Butte',
 'Calaveras',
 'Colusa',
 'Contra Costa',
 'Del Norte',
 'El Dorado',
 'Fresno',
 'Glenn',
 'Humboldt',
 'Imperial',
 'Inyo',
 'Kern',
 'Kings',
 'Lake',
 'Lassen',
 'Los Angeles',
 'Madera',
 'Marin',
 'Mariposa',
 'Mendocino',
 'Merced',
 'Modoc',
 'Mono',
 'Monterey',
 'Napa',
 'Nevada',
 'Orange',
 'Placer',
 'Plumas',
 'Riverside',
 'Sacramento',
 'San Benito',
 'San Bernardino',
 'San Diego',
 'San Francisco',
 'San Joaquin',
 'San Luis Obispo',
 'San Mateo',
 'Santa Barbara',
 'Santa Clara',
 'Santa Cruz',
 'Shasta',
 'Sierra',
 'Siskiyou',
 'Solano',
 'Sonoma',
 'Stanislaus',
 'Sutter',
 'Tehama',
 'Trinity',
 'Tulare',
 'Tuolumne',
 'Ventura',
 'Yolo',
 'Yuba']

In [ ]:
# update_county_incidents(conn, key, "Los Angeles")

except triggered
Page 1 processed.
Page 2 processed.
Page 3 processed.
Page 4 processed.
Page 5 processed.
Page 6 processed.


In [15]:
cmd =\
"""
SELECT name FROM sqlite_master WHERE name='counties'
"""
cursor = conn.cursor()
cursor.execute(cmd)
len(cursor.fetchall())

1

In [18]:
data = get_county_incidents(conn, "los angeles")
td.incident_heatmap(data, radius=10)


            SELECT * FROM incidents
            WHERE lat BETWEEN 32.75004 AND 34.823302
            AND lng BETWEEN -118.951721 AND -117.646374
        
